<a href="https://colab.research.google.com/github/BelideSaiTeja/Sentiment-Analysis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os

2024-03-18 06:47:29.899410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 06:47:29.899465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 06:47:29.900931: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:

!pip install transformers
!pip install tensorflow_datasets

In [ ]:
import pandas as pd
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/review-data/amazon_reviews_us_Mobile_Electronics_v1_00.tsv', sep='\t', on_bad_lines="skip")
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5.0,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5.0,0.0,1.0,N,Y,five star,it's great,2015-08-31
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5.0,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,221169481,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4.0,0.0,0.0,N,Y,Work very well but couldn't get used to not he...,Work very well but couldn't get used to not he...,2015-08-31
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2.0,0.0,0.0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31


In [ ]:
columns_of_interest = ['star_rating', 'review_body']
data = df[columns_of_interest]
data.head()

,star_rating,review_body
0,5.0,"As advertised. Everything works perfectly, I'm..."
1,5.0,it's great
2,5.0,These work great and fit my life proof case fo...
3,4.0,Work very well but couldn't get used to not he...
4,2.0,"Be careful with these products, I have bought ..."


In [ ]:
data.isnull().sum()

star_rating    2
review_body    4
dtype: int64

In [ ]:
data = data.dropna()

In [ ]:
data.shape

(104848, 2)

In [ ]:
data['star_rating'] = data['star_rating'].apply(lambda rating: 'positive' if rating >=3 else 'negative')

In [ ]:
data['sentiment'] = data['star_rating'].map({'positive':1,
                                             'negative':0})

# drop the star_rating column
data.drop(columns = ['star_rating'], inplace = True)

In [ ]:
X = data.drop('sentiment', axis = 1)
y = data['sentiment']

In [ ]:
X = X['review_body'].tolist()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 7)

In [ ]:
# load the tokenizer
from transformers import DistilBertTokenizerFast
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

In [ ]:
print('Sentence: ', X_train[0])
tokenizer(X_train[0],
          truncation = True,
          padding = True,
          max_length = 512,
          return_tensors = 'tf')

Sentence:  All of the negative comments are true!   Clock runs fast, nature sounds are bad, and display is unreadable!


{'input_ids': <tf.Tensor: shape=(1, 26), dtype=int32, numpy=
array([[  101,  2035,  1997,  1996,  4997,  7928,  2024,  2995,   999,
         5119,  3216,  3435,  1010,  3267,  4165,  2024,  2919,  1010,
         1998,  4653,  2003,  4895, 16416, 20782,   999,   102]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 26), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1]], dtype=int32)>}

In [ ]:
max_length = 512
train_encodings = tokenizer(X_train,
                            truncation = True,
                            padding = True,
                            return_tensors = 'tf',
                            max_length = max_length)
test_encodings = tokenizer(X_test,
                            truncation = True,
                            padding = True,
                            return_tensors = 'tf',
                            max_length = max_length)

In [ ]:
# convert tokenizer to tf object
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),
                                                    y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),
                                                    y_test))

In [ ]:
from transformers import TFDistilBertForSequenceClassification
model = TFDistilBertForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path = model_name,
    num_labels = 2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate = 5e-5,
)

In [ ]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer = 'adam', loss = loss_func, metrics = ['accuracy'])

In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0 (unused)
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
batch_size = 16
epochs = 2
model.fit(train_dataset.shuffle(len(X_train)).batch(batch_size),
          epochs = epochs,
          batch_size = batch_size,
          validation_data = test_dataset.shuffle(len(X_test)).batch(batch_size))

Epoch 1/2
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


I0000 00:00:1710744803.374520     187 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5243/5243 [==============================] - 5245s 990ms/step - loss: 0.5533 - accuracy: 0.7630 - val_loss: 0.5516 - val_accuracy: 0.7600
Epoch 2/2
5243/5243 [==============================] - 5187s 989ms/step - loss: 0.5486 - accuracy: 0.7635 - val_loss: 0.5511 - val_accuracy: 0.7600


In [ ]:
model.save_pretrained('/tmp/sentiment_model')

In [ ]:
# preprocess the unseen data
unseen_data = data.sample(frac = 0.05, random_state = 7)
unseen_reviews = unseen_data['review_body'].tolist()

unseen_encodings = tokenizer(unseen_reviews,
                             padding = True,
                             truncation = True,
                             return_tensors = 'tf')
y_unseen = unseen_data['sentiment'].tolist()
unseen_encodings = tf.data.Dataset.from_tensor_slices((dict(unseen_encodings),
                                                       y_unseen))

In [ ]:
model.evaluate(unseen_encodings.shuffle(len(unseen_reviews)).batch(batch_size),
               #retrun_dict = True,
               batch_size = batch_size)

328/328 [==============================] - 109s 318ms/step - loss: 0.5552 - accuracy: 0.7564


[0.5552161335945129, 0.7563906908035278]